In [1]:
import joblib
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, confusion_matrix


In [2]:
lr = joblib.load("logistic_churn_model.pkl")
X_test = joblib.load("X_test.pkl")
y_test = joblib.load("y_test.pkl")

churn_prob = lr.predict_proba(X_test)[:, 1]


In [3]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix
import pandas as pd

thresholds = [0.30, 0.40, 0.50, 0.60]
results = []

for t in thresholds:
    y_pred_t = (churn_prob >= t).astype(int)
    
    precision = precision_score(y_test, y_pred_t)
    recall = recall_score(y_test, y_pred_t)
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_t).ravel()
    
    results.append({
        "Threshold": t,
        "Precision (Churn)": round(precision, 3),
        "Recall (Churn)": round(recall, 3),
        "False Positives": fp,
        "False Negatives": fn
    })

threshold_df = pd.DataFrame(results)
threshold_df


,Threshold,Precision (Churn),Recall (Churn),False Positives,False Negatives
0,0.3,0.500,0.949,355,19
1,0.4,0.532,0.906,298,35
2,0.5,0.576,0.877,241,46
3,0.6,0.639,0.813,172,70


In [4]:
FINAL_THRESHOLD = 0.40


In [5]:
final_prediction = (churn_prob >= FINAL_THRESHOLD).astype(int)

In [7]:
print(final_prediction)

[0 0 0 ... 1 0 1]


In [8]:
decision_df = pd.DataFrame({
    "Actual_Churn": y_test.values,
    "Churn_Probability": churn_prob,
    "Predicted_Churn": final_prediction
})


In [9]:
def risk_segment(prob):
    if prob < 0.30:
        return "Low Risk"
    elif prob < 0.60:
        return "Medium Risk"
    else:
        return "High Risk"

decision_df["Risk_Level"] = decision_df["Churn_Probability"].apply(risk_segment)


In [10]:
decision_df.head()


,Actual_Churn,Churn_Probability,Predicted_Churn,Risk_Level
0,0,0.318649,0,Medium Risk
1,0,0.141215,0,Low Risk
2,0,0.000488,0,Low Risk
3,0,0.000503,0,Low Risk
4,0,0.001211,0,Low Risk
